### Проект N 4
## Рекомендательная система

### Цель: 
Подготовить основу рекомендательной системы, с помощью которой можно будет предлагать пользователям 
интересные курсы и благодаря этому повышать средний чек.

### Задачи:
- Проанализировать данные с помощью SQL и Python 
- Подготовить таблицу, содержащую список курсов и по два рекомендованных курса для каждого из них

                                                                                                            Соколова Юлия SDA_1031


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
from itertools import combinations
import collections
from collections import Counter
import warnings
import matplotlib.pyplot as plt
import psycopg2
import psycopg2.extras 
import seaborn as sns
import random

# Подключаемся к базе данных, выгружая из нее только необходимые нам данные, создаем на их основе датасет

def getData():
    query = '''with more_courses_users as
    (
    SELECT 
    DISTINCT user_id
    FROM final.carts c 
    JOIN final.cart_items ci ON c.id=ci.cart_id
    WHERE c.state='successful' AND ci.resource_type='Course'
    GROUP BY 1
    HAVING COUNT (DISTINCT resource_id) > 1
    ),

    all_purchases as
    (
    SELECT 
    DISTINCT user_id,
    resource_id
    FROM final.carts c 
    left JOIN final.cart_items ci ON c.id=ci.cart_id
    WHERE c.state='successful' AND ci.resource_type='Course'
    )
    SELECT
    *
    FROM more_courses_users mcu
    JOIN all_purchases ap on mcu.user_id=ap.user_id
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

recommendations_df = pd.DataFrame(getData())
recommendations_df.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [2]:
# Группируем курсы по юзеру, выводя только уникальных юзеров в первом столбце и все купленные каждым юзером курсы во втором

df_grouped = recommendations_df.groupby('user_id')['resource_id'].apply(list).reset_index()
df_grouped.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[357, 356]"


In [3]:
# Сортируем курсы в порядке возрастания в каждой строке, чтобы вдальнейшем исключить образование зеркальных пары курсов

df_grouped['resource_id'] = df_grouped['resource_id'].apply(lambda y: sorted(y))

In [4]:
# Объединяем курсы внутри каждой строки в пары, при этом каждый курс образует отдельную пару с каждым следующим курсом в строке

df_grouped['resource_id'] = df_grouped['resource_id'].apply(lambda x: list(itertools.combinations(x,2)))
df_grouped.head()

,user_id,resource_id
0,51,"[(516, 1099)]"
1,6117,"[(356, 357), (356, 1125), (357, 1125)]"
2,10275,"[(553, 1147)]"
3,10457,"[(361, 1138)]"
4,17166,"[(356, 357)]"


In [5]:
# Создаем множество пар для подсчета количества уникальных пар

set_of_pairs = set()
for courses in df_grouped['resource_id']:
    for e in courses:
        set_of_pairs.add(e)

print(len(set_of_pairs))

3989


In [6]:
# Аналогичным образом формируем список пар, а из него словарь, возвращающий уникальные пары курсов в качестве ключей 
# и количество их покупок в качестве значений 

list_of_pairs = []
for courses in df_grouped['resource_id']:
    for e in courses:
        list_of_pairs.append(e)

courses_count = Counter(list_of_pairs)

# Из словаря формируем датасет

courses_count_df = pd.DataFrame(list(courses_count.items()), columns=['pairs','count'])
courses_count_df.head()

,pairs,count
0,"(516, 1099)",25
1,"(356, 357)",100
2,"(356, 1125)",44
3,"(357, 1125)",52
4,"(553, 1147)",16


In [7]:
# Определяем минимальную частоту покупки пар курсов как 60% от покупок всех пар курсов. 
# При выборе 50% от всех курсов минимальная граница составила 3 совместных покупки, что является слишком низкой границей 
# и повышает риск того, что курсы не имеют ничего общего и были куплены вместе из-за индивидуальных потребносте юзера

min_freq = np.percentile(courses_count_df['count'],60)
min_freq

5.0

In [8]:
# Пишем функцию, аргументом которой является id курса, при вводе которого функция возвращает 2 рекомендации, 
# каждая из которых содержит id самого курса, id рекомендуемого курса и частоту совместной покупки, рекомендации возвращаются в порядке убывания частоты

def course_recommendation(course_id):
    rec_list=[]
    for i in courses_count.keys():
        if i[0] == course_id:
            rec_list.append((i, courses_count[i]))
        if i[1] == course_id:
            rec_list.append((i, courses_count[i]))
    rec_list = sorted(rec_list, key=lambda x: x[1],  reverse=True)
    return rec_list[:2]
        
# Проверяем работу рекомендательной функции 
course_recommendation(517)

[((517, 551), 52), ((517, 750), 34)]

In [9]:
# Создаем множество с перечислением уникальных курсов

set_of_courses = set(recommendations_df['resource_id'])
print(len(set_of_courses))

126


In [10]:
# Для курсов, которые продавались в паре с другими курсами менее 5 раз создаем переменную, 
# в которой каждый раз будет выбираться случайный курс из списка всех курсов

random_course = random.choice(list(set_of_courses))

*Если курс редко покупался в паре с любыми другими курсами, то скорее всего этот курс не связан тематически с другими курсами на платформе, или достаточно полноценный и полностью покрывает потребности юзера. Либо наоборот - пользователи не были удовлетворены материалами курса и у них не возникало желания приобретать дальнейшие курсы. Чтобы узнать, соответствует ли какая либо из этих гипотез действительности, можно проанализировать насколько часто пользователи проходят до конца курсы, которые покупались редко в паре с другими курсами*

In [11]:
# Создаем финальную таблицу, в которой в качестве индексов указаны уникальные id курсов, 
# в первом столбце самая частая пара для этого курса, а во втором - вторая по частоте пара, 
# а для курсов, для которых рекомендаций слишком мало, указаны рандомные курсы  

recommendation_list = []
rec_df = pd.DataFrame(recommendation_list, columns=['rec_1', 'rec_2'])
for course_id in set_of_courses:
    rec1 = None
    rec2 = None
    if course_recommendation(course_id)[0][1] >= min_freq:
        rec1 = (set(course_recommendation(course_id)[0][0]) - set([course_id])).pop()
    if course_recommendation(course_id)[1][1] >= min_freq:
        rec2 = (set(course_recommendation(course_id)[1][0]) - set([course_id])).pop()
    if course_recommendation(course_id)[0][1] <= min_freq:
        rec1 = random_course
    if course_recommendation(course_id)[1][1] <= min_freq:
        rec2 = random_course
    rec_df.loc[course_id] = [rec1,rec2]

# Проверяем, для каждого ли курса из списка есть рекомендация     
rec_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 513 to 511
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   rec_1   126 non-null    int64
 1   rec_2   126 non-null    int64
dtypes: int64(2)
memory usage: 3.0 KB


In [12]:
rec_df.head()

,rec_1,rec_2
513,503,551
514,551,515
515,551,489
516,745,553
517,551,750


#### Выводы
*Если посмотреть на процентное соотношение курсов, которые покупались в паре с другими курсами, то можно увидеть, что половина курсов покупалась меньше, чем 3 раза в паре с какими-либо другими курсами. Вероятно у этих курсов нет тематической связи с другими курсами или они самодостаточные. Стоит обратить внимание на эти курсы. Если в целом у них мало продаж, то возможно стоит их актуализировать. Если с продажами таких курсов нет проблем, то возможно стоит рассмотреть создание курсов, близких по тематике. Так же стоит проанализировать поведение пользователей, покупающих такие курсы. Возможно эти курсы достаточно объемные и сложные для восприятия и поэтому занимают много времени у пользователей и в целом являются более продолжительными, длительное время не позволяя пользователям заниматься дополнительными курсами.
Помимо того есть вероятность, что "популярные" курсы сами по себе непродолжительные и достаточно простые, поэтому к ним в дополнение юзеры покупают подобные курсы. Для дальнейшего анализа необходима дополнительная информация о содержании, продолжительности курсов и успешности прохождения их юзерами.*